In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata
import win32com.client as win32

#-----------------EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [2]:
#-----------------TRANSFORM
#adding empty e-mails with the e-mail common pattern
row = df['e_mail'].isnull() & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'


In [3]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata
def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [4]:
df.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,status,status_coverage,benefits_id,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail
0,Stcoop,UNIDADE LONDRINA,THIAGO RODOLFO PEREIRA,13632,11245,69489,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,<NA>,1109,AFM5871,...,ATIVO,ATIVO,24,CAVALO,LC: 100% - CP: 10% - SCANIA - 2011 ATÉ 1980,"De R$100.000,01 até R$125.000,00",S,Cavalo,RenovaÃ§Ã£o,unidade.londrina@transdesk.com.br
1,Stcoop,UNIDADE CUIABA,PRIME ASSISTENCIA VEICULAR LTDA,20356,11247,69507,SEGURO DE VIDA,CRISTIANO MILLER DOS SANTOS,<NA>,<NA>,...,ATIVO,ATIVO,28,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,S,<NA>,Novo,unidade.cuiaba@transdesk.com.br
2,Stcoop,UNIDADE CUIABA,PRIME ASSISTENCIA VEICULAR LTDA,20356,11247,69504,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,<NA>,27641,MCY1375,...,ATIVO,ATIVO,24,3 QUARTOS,LC: 100% - CP: 4% - TODAS AS MARCAS - TODOS OS...,"De R$100.000,01 até R$125.000,00",S,Três Quartos,Novo,unidade.cuiaba@transdesk.com.br
3,Stcoop,UNIDADE MARINGÁ,ADRIANO EVO DE GOIS,9458,11248,69517,SEGURO DE VIDA,ADRIANO EVO DE GOIS,<NA>,<NA>,...,ATIVO,ATIVO,28,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,S,<NA>,RenovaÃ§Ã£o,unidade.maringa@transdesk.com.br
4,Stcoop,UNIDADE MARINGÁ,ADRIANO EVO DE GOIS,9458,11248,69514,REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO,<NA>,1479,ATA2425,...,ATIVO,ATIVO,25,TRUCK / BI-TRUCK,TABELA GERAL,"De R$30.000,01 a R$40.000,00",S,Truck,RenovaÃ§Ã£o,unidade.maringa@transdesk.com.br


base_analise

In [41]:

#excluir coluna de e-mail pois está diferenciando na análise
#df = df.drop('e_mail', axis=1)
df = df.drop_duplicates()

#pegar apenas os conjuntos obrigatorios de seguro de vida
beneficios_conjuntos = df.groupby(['conjunto', 'cooperativa'])['benefits_id'].unique()
conjuntos_validos = beneficios_conjuntos[(beneficios_conjuntos.apply(lambda x: {2,6}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {24,28}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {40,44}.issubset((x))))].index

df_obrigatorio = df[df.set_index(['conjunto','cooperativa']).index.isin(conjuntos_validos)].copy()

#criar colunas para posterior agregação
df_obrigatorio['casco'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x == 2 or x == 24 or x == 40) else pd.NA)
df_obrigatorio['seg_vida'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x==6 or x==28 or x==44) else pd.NA)

#filtrando apenas os que fazem sentido para a análise (casco e seguro de vida)
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

conjuntos_analise

In [42]:


df_analise = df_obrigatorio_filtrado.groupby(['cooperativa','unidade','matricula','conjunto']).agg(
    qtd_seg_vida = ('seg_vida', 'count'),
    qtd_casco = ('casco', 'count'),
    qtd_motorista = ('motorista', 'count')
).reset_index()

df_analise.sort_values(by='conjunto', ascending=False)

df_analysis=df_analise[(df_analise['qtd_motorista']!=df_analise['qtd_casco'])|
                      (df_analise['qtd_motorista']!=df_analise['qtd_seg_vida'])|
                      (df_analise['qtd_casco']!=df_analise['qtd_seg_vida'])]

df_analysis.head()
#df_analysis = df_analise[row, ['cooperativa','unidade','matricula','conjunto','qtd_motoristas','qtd_cascos','qtd_segvida']]

file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
df_analysis.to_excel(file_path, engine = 'openpyxl', index = False)


conjuntos_aberto

In [43]:
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

df_open = df_obrigatorio_filtrado.sort_values(by='conjunto', ascending=False)
#df_open = df_open[['cooperativa','unidade', 'cliente', 'matricula','conjunto','coverage','beneficio', 'motorista', 'placa', 'casco', 'seg_vida', 'tipo_veiculo', 'benefits_id']] 
file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_aberto.xlsx'
df_open.to_excel(file_path, engine = 'openpyxl', index = False)

In [45]:
print(
    df_open['motorista'].count(),
    df_open['casco'].count(),
    df_open['seg_vida'].count()
)

10729 10961 10729


In [ ]:
#-----------------LOAD
#Function to send e-mails

def enviar_email(p_email, p_unidade, p_placas, p_motoristas):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail - ALTEREI PARA TESTE
    email.To = "dados13@grupounus.com.br; dados03@grupounus.com.br"
    email.Subject = "[CADASTRAMENTO DE MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""
    <p>Prezado(a),<p/>

    <p>A unidade {p_unidade}, correspondente ao e-mail {p_email}, possui pendências de cadastramento de motoristas por placa:

    <p>Número de placas cadastradas: {p_placas}
    Número de motoristas cadastrados: {p_motoristas}<p/>

    <p>Favor, cadastrar as placas remanescentes para cômputo na base de dados da empresa, obrigado!<p/>

    <p>Atenciosamente,<p/>

    <p>Equipe Análise de Dados - Grupo Unus<p/>

    <p>(<i>Este é um e-mail automático, por favor não responda<i/>)<p/>
 
    """

    email.Send()

    print("Email enviado")

In [ ]:

df_numeros = df_unidemail_filtered.groupby(['unidade']).agg(
    qtd_placas=('placas', 'sum'),
    qtd_motoristas=('motoristas', 'sum')
).reset_index()

for idx, row in df_unidemail_filtered_test.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']

    unidade_info = df_numeros.loc[df_numeros['unidade'] == iUnidade].iloc[0]

    iQtdPlacas = unidade_info['qtd_placas']
    iQtdMotoristas = unidade_info['qtd_motoristas']

    # if __name__=="__main__":
    #     enviar_email(iEmail, iUnidade, iQtdPlacas, iQtdMotoristas)

    
    print(f"e_mail:ENVIADO PRA GENTE TESTE/unidade:{iUnidade}/placas:{iQtdPlacas}/motoristas:{iQtdMotoristas}")


        

Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:TS CONSULTORIA EM TRANSPORTES LTDA/placas:939/motoristas:218
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE CAMPO GRANDE/placas:2123/motoristas:110


In [ ]:


path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)